In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC

from sklearn.decomposition import PCA

from utils.data import CLASS_NAMES, read_dataset
from utils.image_preprocessing import preprocess_dataset
from utils.hyperparameter_optimizer import optimize_hyperparameters

%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 5]

# Loading data

In [ ]:
X_raw, y = read_dataset('./data/rockpaperscissors/')
X = preprocess_dataset(X_raw, image_shape=(300,200))

In [3]:
X.shape

(2188, 60000)

In [4]:
TRAIN_SIZE = int(0.8 * y.shape[0])
X_train, y_train = X[:TRAIN_SIZE], y[:TRAIN_SIZE]
X_test, y_test = X[TRAIN_SIZE:], y[TRAIN_SIZE:]

In [5]:
X_train.shape

(1750, 60000)

### Scaled images

In [6]:
X_scaled = preprocess_dataset(X_raw, image_shape=(150, 100))
X_scaled_train, X_scaled_test = X_scaled[:TRAIN_SIZE], X_scaled[TRAIN_SIZE:]

In [7]:
X_scaled.shape

(2188, 15000)

### PCA

In [8]:
pca_model = PCA(n_components=300)
X_pca = pca_model.fit_transform(X)

X_pca_train = X_pca[:TRAIN_SIZE]
X_pca_test = X_pca[TRAIN_SIZE:]

In [9]:
X_pca.shape

(2188, 300)

# Training models

## Hyperparameter tuning

For each model we select hyperparameters that maximize mean accuracy measured by cross-validation on training set.

### KNN

#### Full images

In [10]:
knn_result = optimize_hyperparameters('knn', X_train, y_train, n_trials=100)
print(knn_result)

[I 2024-02-08 15:14:36,809] A new study created in memory with name: no-name-93ce20ac-defb-410d-9662-df543e81ec83
[I 2024-02-08 15:14:49,214] Trial 9 finished with value: 0.8222857142857144 and parameters: {'n_neighbors': 49}. Best is trial 9 with value: 0.8222857142857144.
[I 2024-02-08 15:14:57,158] Trial 10 finished with value: 0.8542857142857143 and parameters: {'n_neighbors': 28}. Best is trial 10 with value: 0.8542857142857143.
[I 2024-02-08 15:14:57,557] Trial 1 finished with value: 0.8742857142857143 and parameters: {'n_neighbors': 18}. Best is trial 1 with value: 0.8742857142857143.
[I 2024-02-08 15:14:59,656] Trial 2 finished with value: 0.905142857142857 and parameters: {'n_neighbors': 5}. Best is trial 2 with value: 0.905142857142857.
[I 2024-02-08 15:14:59,980] Trial 5 finished with value: 0.8571428571428571 and parameters: {'n_neighbors': 26}. Best is trial 2 with value: 0.905142857142857.
[I 2024-02-08 15:15:02,805] Trial 7 finished with value: 0.8805714285714286 and par

FrozenTrial(number=21, state=1, values=[0.9125714285714286], datetime_start=datetime.datetime(2024, 2, 8, 15, 15, 6, 40561), datetime_complete=datetime.datetime(2024, 2, 8, 15, 15, 28, 234207), params={'n_neighbors': 3}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_neighbors': IntDistribution(high=49, log=False, low=3, step=1)}, trial_id=21, value=None)


In [11]:
knn_result.params

{'n_neighbors': 3}

In [12]:
knn_result.value

0.9125714285714286

In [13]:
best_knn_model = KNeighborsClassifier(n_jobs=-1, **knn_result.params)
best_knn_model.fit(X_train, y_train)
best_knn_model.score(X_test, y_test)

0.9041095890410958

#### Scaled images

In [14]:
knn_scaled_result = optimize_hyperparameters('knn', X_scaled_train, y_train, n_trials=100)
print(knn_scaled_result)

[I 2024-02-08 15:18:05,390] A new study created in memory with name: no-name-87ce219a-7d2e-4381-8f35-447717d482ad
[I 2024-02-08 15:18:06,310] Trial 0 finished with value: 0.8885714285714286 and parameters: {'n_neighbors': 14}. Best is trial 0 with value: 0.8885714285714286.
[I 2024-02-08 15:18:06,985] Trial 2 finished with value: 0.8788571428571428 and parameters: {'n_neighbors': 19}. Best is trial 0 with value: 0.8885714285714286.
[I 2024-02-08 15:18:07,529] Trial 1 finished with value: 0.8657142857142857 and parameters: {'n_neighbors': 24}. Best is trial 0 with value: 0.8885714285714286.
[I 2024-02-08 15:18:07,944] Trial 3 finished with value: 0.8777142857142858 and parameters: {'n_neighbors': 18}. Best is trial 0 with value: 0.8885714285714286.
[I 2024-02-08 15:18:08,432] Trial 4 finished with value: 0.8514285714285714 and parameters: {'n_neighbors': 32}. Best is trial 0 with value: 0.8885714285714286.
[I 2024-02-08 15:18:08,962] Trial 5 finished with value: 0.8794285714285716 and p

FrozenTrial(number=22, state=1, values=[0.9154285714285715], datetime_start=datetime.datetime(2024, 2, 8, 15, 18, 11, 464627), datetime_complete=datetime.datetime(2024, 2, 8, 15, 18, 17, 566998), params={'n_neighbors': 3}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_neighbors': IntDistribution(high=49, log=False, low=3, step=1)}, trial_id=22, value=None)


In [15]:
knn_scaled_result.params

{'n_neighbors': 3}

In [16]:
knn_scaled_result.value

0.9154285714285715

In [17]:
best_knn_model_scaled = KNeighborsClassifier(n_jobs=-1, **knn_scaled_result.params)
best_knn_model_scaled.fit(X_scaled_train, y_train)
best_knn_model_scaled.score(X_scaled_test, y_test)

0.906392694063927

#### PCA

In [18]:
knn_pca_result = optimize_hyperparameters('knn', X_pca_train, y_train, n_trials=100)
print(knn_pca_result)

[I 2024-02-08 15:18:59,209] A new study created in memory with name: no-name-06bc29f9-fe7d-4b75-9db3-71bfd9233fd1
[I 2024-02-08 15:18:59,369] Trial 4 finished with value: 0.8411428571428573 and parameters: {'n_neighbors': 38}. Best is trial 4 with value: 0.8411428571428573.
[I 2024-02-08 15:18:59,372] Trial 8 finished with value: 0.8588571428571429 and parameters: {'n_neighbors': 29}. Best is trial 8 with value: 0.8588571428571429.
[I 2024-02-08 15:18:59,378] Trial 9 finished with value: 0.8651428571428571 and parameters: {'n_neighbors': 24}. Best is trial 9 with value: 0.8651428571428571.
[I 2024-02-08 15:18:59,392] Trial 5 finished with value: 0.8314285714285715 and parameters: {'n_neighbors': 42}. Best is trial 9 with value: 0.8651428571428571.
[I 2024-02-08 15:18:59,405] Trial 1 finished with value: 0.9040000000000001 and parameters: {'n_neighbors': 9}. Best is trial 1 with value: 0.9040000000000001.
[I 2024-02-08 15:18:59,423] Trial 3 finished with value: 0.9062857142857143 and pa

FrozenTrial(number=27, state=1, values=[0.9194285714285714], datetime_start=datetime.datetime(2024, 2, 8, 15, 18, 59, 602355), datetime_complete=datetime.datetime(2024, 2, 8, 15, 18, 59, 722642), params={'n_neighbors': 3}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_neighbors': IntDistribution(high=49, log=False, low=3, step=1)}, trial_id=27, value=None)


In [19]:
knn_pca_result.params

{'n_neighbors': 3}

In [20]:
knn_pca_result.value

0.9194285714285714

In [21]:
best_knn_model_pca = KNeighborsClassifier(n_jobs=-1, **knn_pca_result.params)
best_knn_model_pca.fit(X_pca_train, y_train)
best_knn_model_pca.score(X_pca_test, y_test)

0.910958904109589

### Decision Tree

### Random Forest

### SVM

### XGBoost